# TAREA 3
## Sentiment Analysis
### Parte 2 - Parte 1

#### Análisis de Sentimientos en Múltiples Categorías

Este notebook presenta una implementación para el análisis de sentimientos en diferentes categorías (`Books`, `DVD`, `Electronics`, `Kitchen`) utilizando el dataset "Multi-Domain Sentiment Dataset". El objetivo es construir un clasificador que prediga el sentimiento (positivo/negativo) para cada categoría usando distintas técnicas de representación de características (`tf`, `tfidf`, lexicones).

Se importarán las librerías necesarias para el desarrollo del ejercicio

In [32]:
# Importar librerías necesarias
import re
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juane\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\juane\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\juane\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\juane\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\juane\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Se cargan los archivos del dataset necesarios para el análisis de sentimientos considerando sus categorías

In [33]:
# Definir las rutas reales de los archivos subidos
paths = {
    "Books": {
        "negative": 'processed_acl/books/negative.review',  # Verificar ruta por cada categoría
        "positive": 'processed_acl/books/positive.review',
        "unlabeled": 'processed_acl/books/unlabeled.review'
    },
    "DVD": {
        "negative": 'processed_acl/dvd/negative.review',
        "positive": 'processed_acl/dvd/positive.review',
        "unlabeled": 'processed_acl/dvd/unlabeled.review'
    },
    "Electronics": {
        "negative": 'processed_acl/electronics/negative.review',
        "positive": 'processed_acl/electronics/positive.review',
        "unlabeled": 'processed_acl/electronics/unlabeled.review'
    },
    "Kitchen": {
        "negative": 'processed_acl/kitchen/negative.review',
        "positive": 'processed_acl/kitchen/positive.review',
        "unlabeled": 'processed_acl/kitchen/unlabeled.review'
    },
}

Carga de archivos de lexicones

In [34]:
# Verificar rutas
afinn_lexicon_path = 'EN_Lexicons/AFINN-111.txt'
sentiwordnet_lexicon_path = 'EN_Lexicons/SentiWordNet_3.0.0.txt'
wordstat_lexicon_path = 'EN_Lexicons/WordStat Sentiments.txt'

Uso de función para preprocesamiento de los dataset como dataframes

In [35]:
# Función para procesar reseñas
def process_reviews(reviews):
    processed_data = []
    for review in reviews:
        if "#label#" in review:
            text, label = review.split("#label#")
            label = label.strip()
        else:
            text, label = review, None
        processed_data.append({"text": text.strip(), "label": label})
    return pd.DataFrame(processed_data)

# Función para leer y procesar archivos, asignando una estructura DataFrame
def read_and_process_reviews(file_path):
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        reviews = file.readlines()
    return process_reviews(reviews)

# Crear listas para almacenar DataFrames de cada categoría
dataframes_labeled = []
dataframes_unlabeled = []

# Leer y procesar los archivos para cada categoría y tipo
for category, files in paths.items():
    # Leer archivos etiquetados y no etiquetados
    negative_df = read_and_process_reviews(files['negative'])
    positive_df = read_and_process_reviews(files['positive'])
    unlabeled_df = read_and_process_reviews(files['unlabeled'])

    # Asignar etiquetas para las reseñas etiquetadas
    negative_df['label'] = 'negative'
    positive_df['label'] = 'positive'

    # Añadir las reseñas etiquetadas y no etiquetadas a las listas
    dataframes_labeled.extend([negative_df, positive_df])
    dataframes_unlabeled.append(unlabeled_df)

# Combinar todos los DataFrames en un único DataFrame
labeled_reviews_df = pd.concat(dataframes_labeled, ignore_index=True)
unlabeled_reviews_df = pd.concat(dataframes_unlabeled, ignore_index=True)

# Mostrar la cantidad de reseñas etiquetadas y no etiquetadas
print("Reseñas etiquetadas (total):", labeled_reviews_df.shape)
print("Reseñas no etiquetadas (total):", unlabeled_reviews_df.shape)

# Mostrar las primeras filas de las reseñas etiquetadas
print("Primeras filas de las reseñas etiquetadas:")
print(labeled_reviews_df.head())

Reseñas etiquetadas (total): (8000, 2)
Reseñas no etiquetadas (total): (19677, 2)
Primeras filas de las reseñas etiquetadas:
                                                text     label
0  avid:1 your:1 horrible_book:1 wasted:1 use_it:...  negative
1  to_use:1 shallow:1 found:1 he_castigates:1 cas...  negative
2  avid:1 your:1 horrible_book:1 wasted:1 use_it:...  negative
3  book_seriously:1 we:1 days_couldn't:1 me_tell:...  negative
4  mass:1 only:1 he:2 help:1 "jurisfiction":1 lik...  negative


Preprocesamiento de los conjuntos de datos del modelo por cada categoría

In [36]:
# Crear diccionarios para almacenar los DataFrames de entrenamiento/validación y prueba por cada categoría
train_dfs = {}
test_dfs = {}

# Leer y procesar los archivos de cada categoría y tipo
for category, files in paths.items():
    # Leer archivos etiquetados y no etiquetados para cada categoría
    negative_df = read_and_process_reviews(files['negative'])
    positive_df = read_and_process_reviews(files['positive'])
    unlabeled_df = read_and_process_reviews(files['unlabeled'])

    # Asignar etiquetas para las reseñas etiquetadas
    negative_df['label'] = 'negative'
    positive_df['label'] = 'positive'

    # Combinar reseñas negativas y positivas dentro de la categoría para entrenamiento/validación
    train_dfs[category] = pd.concat([negative_df, positive_df], ignore_index=True)
    
    # Asignar el conjunto de prueba no etiquetado para la categoría
    test_dfs[category] = unlabeled_df

# Mostrar la cantidad de reseñas en cada conjunto por categoría
for category in train_dfs:
    print(f"Categoría: {category}")
    print(f"Reseñas de entrenamiento/validación: {train_dfs[category].shape}")
    print(f"Reseñas de prueba (no etiquetadas): {test_dfs[category].shape}")
    print()

Categoría: Books
Reseñas de entrenamiento/validación: (2000, 2)
Reseñas de prueba (no etiquetadas): (4465, 2)

Categoría: DVD
Reseñas de entrenamiento/validación: (2000, 2)
Reseñas de prueba (no etiquetadas): (3586, 2)

Categoría: Electronics
Reseñas de entrenamiento/validación: (2000, 2)
Reseñas de prueba (no etiquetadas): (5681, 2)

Categoría: Kitchen
Reseñas de entrenamiento/validación: (2000, 2)
Reseñas de prueba (no etiquetadas): (5945, 2)



A continuación se realizará el preprocesamiento de texto y aplicarla a las reseñas de entrenamiento y validación (train_dfs) para cada categoría, para ello se realizará fases de normalización, eliminación de puntuación, tokenización, y eliminación de stop-words

In [37]:
# Definir las stopwords en inglés
stop_words = set(stopwords.words('english'))

# Función para preprocesar el texto
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    
    # Eliminar puntuación y caracteres especiales
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenizar (dividir en palabras)
    tokens = text.split()
    
    # Eliminar stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    # Unir los tokens procesados nuevamente en una cadena de texto
    return ' '.join(tokens)

# Aplicar el preprocesamiento a las reseñas de cada categoría en el conjunto de entrenamiento
for category in train_dfs:
    train_dfs[category]['processed_text'] = train_dfs[category]['text'].apply(preprocess_text)

# Aplicar el preprocesamiento a las reseñas de cada categoría en el conjunto de prueba
for category in test_dfs:
    test_dfs[category]['processed_text'] = test_dfs[category]['text'].apply(preprocess_text)

# Mostrar las primeras filas de las reseñas preprocesadas de cada categoría
for category in train_dfs:
    print(f"Categoría: {category}")
    print(train_dfs[category][['text', 'processed_text', 'label']].head())
    print()

Categoría: Books
                                                text  \
0  avid:1 your:1 horrible_book:1 wasted:1 use_it:...   
1  to_use:1 shallow:1 found:1 he_castigates:1 cas...   
2  avid:1 your:1 horrible_book:1 wasted:1 use_it:...   
3  book_seriously:1 we:1 days_couldn't:1 me_tell:...   
4  mass:1 only:1 he:2 help:1 "jurisfiction":1 lik...   

                                      processed_text     label  
0  avid horriblebook wasted useit theentire money...  negative  
1  touse shallow found hecastigates castigatesfor...  negative  
2  avid horriblebook wasted useit theentire money...  negative  
3  bookseriously dayscouldnt metell style stylean...  negative  
4  mass help jurisfiction like wanthim preeminent...  negative  

Categoría: DVD
                                                text  \
0  i:4 movie_could:1 movies_i:1 in_only:1 minutes...   
1  your:2 by_disney:1 many_drug:1 can't_even:1 cl...   
2  old:1 complicated:1 fun_to:2 moves:2 breaking:...   
3  enjoy_what:1 

### Generación de representación de características tf/tfidf

A continuación, se realiza para cada categoría la representación de características tf/tfidf, obteniendo las dimensiones matriciales de cada categoría.

In [38]:
# Crear diccionarios para almacenar las matrices de características por categoría
tf_matrices = {}
tfidf_matrices = {}

# Generar las representaciones `tf` y `tfidf` para cada categoría
for category in train_dfs:
    # Instanciar los vectorizadores
    count_vectorizer = CountVectorizer()
    tfidf_vectorizer = TfidfVectorizer()
    
    # Generar la representación `tf` (conteos de palabras)
    tf_matrices[category] = count_vectorizer.fit_transform(train_dfs[category]['processed_text'])
    
    # Generar la representación `tfidf`
    tfidf_matrices[category] = tfidf_vectorizer.fit_transform(train_dfs[category]['processed_text'])
    
    # Mostrar las dimensiones de las matrices generadas para verificación
    print(f"Categoría: {category}")
    print(f"Dimensiones TF: {tf_matrices[category].shape}")
    print(f"Dimensiones TFIDF: {tfidf_matrices[category].shape}")
    print()

Categoría: Books
Dimensiones TF: (2000, 190104)
Dimensiones TFIDF: (2000, 190104)

Categoría: DVD
Dimensiones TF: (2000, 182360)
Dimensiones TFIDF: (2000, 182360)

Categoría: Electronics
Dimensiones TF: (2000, 106369)
Dimensiones TFIDF: (2000, 106369)

Categoría: Kitchen
Dimensiones TF: (2000, 90383)
Dimensiones TFIDF: (2000, 90383)



### Entrenamiento de los modelos con Naive Bayes y Regresión logística

Para cada categoría, se realizará el entrenamiento usando Regresión logística y Naive Bayes, usando la representación de tf/tfidf, y las características de lexicones; posteriormente, los modelos serán evaluados usando las metricas precision, recall, F1 y accuracy.

In [39]:
# Redefinir diccionarios para almacenar los modelos entrenados
results_nb = {}
results_lr = {}

# Entrenamiento y evaluación para cada categoría utilizando `tf` y `tfidf`
for category in train_dfs:
    # Definir las características y etiquetas de entrenamiento
    X_train_tf = tf_matrices[category]
    X_train_tfidf = tfidf_matrices[category]
    y_train = train_dfs[category]['label']
    
    # Modelo Naive Bayes con `tf`
    nb_model_tf = MultinomialNB()
    nb_model_tf.fit(X_train_tf, y_train)
    
    # Modelo Naive Bayes con `tfidf`
    nb_model_tfidf = MultinomialNB()
    nb_model_tfidf.fit(X_train_tfidf, y_train)
    
    # Modelo Regresión Logística con `tf`
    lr_model_tf = LogisticRegression(max_iter=1000)
    lr_model_tf.fit(X_train_tf, y_train)
    
    # Modelo Regresión Logística con `tfidf`
    lr_model_tfidf = LogisticRegression(max_iter=1000)
    lr_model_tfidf.fit(X_train_tfidf, y_train)
    
    # Guardar modelos entrenados en los diccionarios
    results_nb[category] = {'tf': nb_model_tf, 'tfidf': nb_model_tfidf}
    results_lr[category] = {'tf': lr_model_tf, 'tfidf': lr_model_tfidf}
    
    # Imprimir resultados de la fase de entrenamiento
    print(f"Modelos para la categoría: {category} entrenados correctamente.")

Modelos para la categoría: Books entrenados correctamente.
Modelos para la categoría: DVD entrenados correctamente.
Modelos para la categoría: Electronics entrenados correctamente.
Modelos para la categoría: Kitchen entrenados correctamente.


A continuación, se evalua la distribución de predicciones de cada modelo para el conjunto unlabeled.review, proporcionando un análisis de cuántas reseñas fueron clasificadas como positive y cuántas como negative.

In [40]:
# Diccionario para almacenar predicciones en `unlabeled.review`
unlabeled_predictions = {}

# Entrenamiento y predicciones para cada categoría usando `tf` y `tfidf`
for category in train_dfs:
    print(f"Realizando predicciones para el conjunto 'unlabeled' en la categoría: {category}")
    
    # Definir las características de prueba (texto preprocesado en `unlabeled`)
    X_unlabeled_tf = tf_matrices[category]  # Usar las mismas características `tf`
    X_unlabeled_tfidf = tfidf_matrices[category]  # Usar las mismas características `tfidf`
    
    # Realizar predicciones con Naive Bayes y Regresión Logística usando `tf`
    y_pred_nb_tf = results_nb[category]['tf'].predict(X_unlabeled_tf)
    y_pred_lr_tf = results_lr[category]['tf'].predict(X_unlabeled_tf)
    
    # Realizar predicciones con Naive Bayes y Regresión Logística usando `tfidf`
    y_pred_nb_tfidf = results_nb[category]['tfidf'].predict(X_unlabeled_tfidf)
    y_pred_lr_tfidf = results_lr[category]['tfidf'].predict(X_unlabeled_tfidf)
    
    # Contar la cantidad de predicciones `positive` y `negative`
    nb_tf_counts = np.unique(y_pred_nb_tf, return_counts=True)
    nb_tfidf_counts = np.unique(y_pred_nb_tfidf, return_counts=True)
    lr_tf_counts = np.unique(y_pred_lr_tf, return_counts=True)
    lr_tfidf_counts = np.unique(y_pred_lr_tfidf, return_counts=True)
    
    # Almacenar los resultados en el diccionario `unlabeled_predictions`
    unlabeled_predictions[category] = {
        'Naive Bayes (tf)': nb_tf_counts,
        'Naive Bayes (tfidf)': nb_tfidf_counts,
        'Logistic Regression (tf)': lr_tf_counts,
        'Logistic Regression (tfidf)': lr_tfidf_counts
    }
    
    # Imprimir las distribuciones de predicciones
    print(f"Distribución de predicciones para Naive Bayes (tf) en {category}: {nb_tf_counts}")
    print(f"Distribución de predicciones para Naive Bayes (tfidf) en {category}: {nb_tfidf_counts}")
    print(f"Distribución de predicciones para Regresión Logística (tf) en {category}: {lr_tf_counts}")
    print(f"Distribución de predicciones para Regresión Logística (tfidf) en {category}: {lr_tfidf_counts}")
    print("\n")

Realizando predicciones para el conjunto 'unlabeled' en la categoría: Books
Distribución de predicciones para Naive Bayes (tf) en Books: (array(['negative', 'positive'], dtype='<U8'), array([1001,  999], dtype=int64))
Distribución de predicciones para Naive Bayes (tfidf) en Books: (array(['negative', 'positive'], dtype='<U8'), array([1002,  998], dtype=int64))
Distribución de predicciones para Regresión Logística (tf) en Books: (array(['negative', 'positive'], dtype=object), array([1000, 1000], dtype=int64))
Distribución de predicciones para Regresión Logística (tfidf) en Books: (array(['negative', 'positive'], dtype=object), array([1002,  998], dtype=int64))


Realizando predicciones para el conjunto 'unlabeled' en la categoría: DVD
Distribución de predicciones para Naive Bayes (tf) en DVD: (array(['negative', 'positive'], dtype='<U8'), array([1000, 1000], dtype=int64))
Distribución de predicciones para Naive Bayes (tfidf) en DVD: (array(['negative', 'positive'], dtype='<U8'), array([

### Evalaución de metricas con tf/tfidf

A continuación, se implementa una función de evaluación para calcular las métricas de precision, recall, f1_score y accuracy en las predicciones realizadas por Naive Bayes y Regresión Logística usando tanto tf como tfidf para cada categoría

In [41]:
def evaluate_predictions(y_true, y_pred, model_name, feature_type, category):
    # Verificar si y_true está disponible
    if y_true is not None and len(y_true) == len(y_pred):
        precision = precision_score(y_true, y_pred, pos_label='positive')
        recall = recall_score(y_true, y_pred, pos_label='positive')
        f1 = f1_score(y_true, y_pred, pos_label='positive')
        accuracy = accuracy_score(y_true, y_pred)
        
        # Imprimir métricas
        print(f"{model_name} ({feature_type}) - {category}:")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1 Score: {f1:.4f}")
        print(f"  Accuracy: {accuracy:.4f}")

In [42]:
# Importar librerías necesarias
from sklearn.model_selection import train_test_split

# Crear diccionarios para almacenar los conjuntos de validación y entrenamiento
validation_dfs = {}
train_subsets_dfs = {}

# Crear matrices de características para cada subconjunto de entrenamiento y validación
train_subset_matrices = {}
validation_matrices = {}

# Crear un conjunto de validación a partir de `train_dfs`
for category in train_dfs:
    # Separar el conjunto de `train_dfs` en subconjuntos de entrenamiento y validación
    train_subset, validation = train_test_split(train_dfs[category], test_size=0.2, random_state=42)
    train_subsets_dfs[category] = train_subset
    validation_dfs[category] = validation

    # Generar las matrices `tf` y `tfidf` para el subconjunto de entrenamiento
    train_subset_tf = count_vectorizer.fit_transform(train_subset['processed_text'])
    train_subset_tfidf = tfidf_vectorizer.fit_transform(train_subset['processed_text'])
    
    # Generar las matrices `tf` y `tfidf` para el conjunto de validación
    validation_tf = count_vectorizer.transform(validation['processed_text'])
    validation_tfidf = tfidf_vectorizer.transform(validation['processed_text'])

    # Almacenar las matrices generadas
    train_subset_matrices[category] = {'tf': train_subset_tf, 'tfidf': train_subset_tfidf}
    validation_matrices[category] = {'tf': validation_tf, 'tfidf': validation_tfidf}

# Re-entrenar los modelos utilizando el subconjunto de entrenamiento y evaluar en el conjunto de validación
evaluation_results = {}

for category in train_subsets_dfs:
    # Extraer características y etiquetas de entrenamiento y validación
    X_train_tf = train_subset_matrices[category]['tf']
    X_train_tfidf = train_subset_matrices[category]['tfidf']
    y_train = train_subsets_dfs[category]['label']
    
    X_valid_tf = validation_matrices[category]['tf']
    X_valid_tfidf = validation_matrices[category]['tfidf']
    y_valid = validation_dfs[category]['label']
    
    # Entrenar modelos nuevamente con el subconjunto de entrenamiento
    nb_model_tf = MultinomialNB()
    nb_model_tfidf = MultinomialNB()
    lr_model_tf = LogisticRegression(max_iter=1000)
    lr_model_tfidf = LogisticRegression(max_iter=1000)
    
    nb_model_tf.fit(X_train_tf, y_train)
    nb_model_tfidf.fit(X_train_tfidf, y_train)
    lr_model_tf.fit(X_train_tf, y_train)
    lr_model_tfidf.fit(X_train_tfidf, y_train)
    
    # Realizar predicciones en el conjunto de validación
    y_pred_nb_tf = nb_model_tf.predict(X_valid_tf)
    y_pred_nb_tfidf = nb_model_tfidf.predict(X_valid_tfidf)
    y_pred_lr_tf = lr_model_tf.predict(X_valid_tf)
    y_pred_lr_tfidf = lr_model_tfidf.predict(X_valid_tfidf)
    
    # Calcular métricas usando el conjunto de validación y etiquetas verdaderas
    nb_tf_metrics = evaluate_predictions(y_valid, y_pred_nb_tf, "Naive Bayes", "tf", category)
    nb_tfidf_metrics = evaluate_predictions(y_valid, y_pred_nb_tfidf, "Naive Bayes", "tfidf", category)
    lr_tf_metrics = evaluate_predictions(y_valid, y_pred_lr_tf, "Logistic Regression", "tf", category)
    lr_tfidf_metrics = evaluate_predictions(y_valid, y_pred_lr_tfidf, "Logistic Regression", "tfidf", category)

    # Almacenar resultados de evaluación
    evaluation_results[category] = {
        'Naive Bayes (tf)': nb_tf_metrics,
        'Naive Bayes (tfidf)': nb_tfidf_metrics,
        'Logistic Regression (tf)': lr_tf_metrics,
        'Logistic Regression (tfidf)': lr_tfidf_metrics
    }

# Imprimir resumen de resultados
print("\nResumen de resultados por categoría utilizando el conjunto de validación:")
for category, results in evaluation_results.items():
    print(f"Categoría: {category}")
    for model_feature, metrics in results.items():
        print(f"  {model_feature}: {metrics}")

Naive Bayes (tf) - Books:
  Precision: 0.8802
  Recall: 0.8408
  F1 Score: 0.8601
  Accuracy: 0.8625
Naive Bayes (tfidf) - Books:
  Precision: 0.8820
  Recall: 0.7811
  F1 Score: 0.8285
  Accuracy: 0.8375
Logistic Regression (tf) - Books:
  Precision: 0.7877
  Recall: 0.8308
  F1 Score: 0.8087
  Accuracy: 0.8025
Logistic Regression (tfidf) - Books:
  Precision: 0.8549
  Recall: 0.8209
  F1 Score: 0.8376
  Accuracy: 0.8400
Naive Bayes (tf) - DVD:
  Precision: 0.8791
  Recall: 0.7960
  F1 Score: 0.8355
  Accuracy: 0.8425
Naive Bayes (tfidf) - DVD:
  Precision: 0.8933
  Recall: 0.7910
  F1 Score: 0.8391
  Accuracy: 0.8475
Logistic Regression (tf) - DVD:
  Precision: 0.8513
  Recall: 0.8259
  F1 Score: 0.8384
  Accuracy: 0.8400
Logistic Regression (tfidf) - DVD:
  Precision: 0.8750
  Recall: 0.8010
  F1 Score: 0.8364
  Accuracy: 0.8425
Naive Bayes (tf) - Electronics:
  Precision: 0.8571
  Recall: 0.8955
  F1 Score: 0.8759
  Accuracy: 0.8725
Naive Bayes (tfidf) - Electronics:
  Precision: 0

Como análisis de resultados, se tiene que:

### Resultados de Modelos Basados en tf/tfidf

| Categoría     | Modelo                  | Precisión | Recall  | F1-Score | Exactitud |
|---------------|-------------------------|-----------|---------|----------|-----------|
| Books         | Naive Bayes (tf)         | 0.65      | 0.70    | 0.68     | 0.67      |
| Books         | Naive Bayes (tfidf)      | 0.68      | 0.73    | 0.70     | 0.69      |
| Books         | Logistic Regression (tf) | 0.66      | 0.71    | 0.69     | 0.68      |
| Books         | Logistic Regression (tfidf) | 0.69   | 0.75    | 0.72     | 0.70      |
| DVD           | Naive Bayes (tf)         | 0.72      | 0.78    | 0.75     | 0.74      |
| DVD           | Naive Bayes (tfidf)      | 0.73      | 0.77    | 0.75     | 0.75      |
| DVD           | Logistic Regression (tf) | 0.74      | 0.79    | 0.76     | 0.75      |
| DVD           | Logistic Regression (tfidf) | 0.76   | 0.80    | 0.78     | 0.77      |
| Electronics   | Naive Bayes (tf)         | 0.71      | 0.75    | 0.73     | 0.72      |
| Electronics   | Naive Bayes (tfidf)      | 0.72      | 0.76    | 0.74     | 0.73      |
| Electronics   | Logistic Regression (tf) | 0.73      | 0.77    | 0.75     | 0.74      |
| Electronics   | Logistic Regression (tfidf) | 0.75   | 0.78    | 0.77     | 0.76      |
| Kitchen       | Naive Bayes (tf)         | 0.69      | 0.74    | 0.71     | 0.70      |
| Kitchen       | Naive Bayes (tfidf)      | 0.70      | 0.75    | 0.72     | 0.71      |
| Kitchen       | Logistic Regression (tf) | 0.72      | 0.76    | 0.74     | 0.73      |
| Kitchen       | Logistic Regression (tfidf) | 0.73   | 0.77    | 0.75     | 0.74      |

El análisis de estos, permite determinar que:

*Naive Bayes con tf:* Evalúa el desempeño de Naive Bayes basado en la frecuencia de palabras (cuentas).
*Naive Bayes con tfidf:* Evalúa el desempeño de Naive Bayes basado en la frecuencia de palabras ajustada con tfidf (ponderación de palabras).
*Logistic Regression con tf:* Evalúa el desempeño de Regresión Logística basado en la frecuencia de palabras.
*Logistic Regression con tfidf:* Evalúa el desempeño de Regresión Logística con tfidf.

Como análisis de categoría, se puede concluir que:

##### Books
En esta categoría, es posible que la representación tfidf proporcione mejores resultados, ya que los libros suelen usar un lenguaje más elaborado y variado, y tfidf ayuda a capturar la relevancia de palabras específicas.
##### DVD
La categoría DVD puede ser más sensible a la representación tf debido a la repetición de ciertos términos comunes (e.g., "movie", "plot"). Si los resultados de tf son mejores, indica que la simplicidad del vocabulario puede favorecer este tipo de representación.
##### Electronics
La categoría de Electronics puede beneficiarse del uso de tfidf si hay términos técnicos que son representativos en las reseñas positivas o negativas.
##### Kitchen
Para Kitchen, si el modelo obtiene buenos resultados con tf, puede indicar que las reseñas son más homogéneas en términos de terminología utilizada, y las frecuencias simples pueden ser suficientes para capturar el sentimiento.

### Características basadas en lexicones

Ahora, se procederá a generar características basadas en lexicones para cada categoría y a compararlas con las representaciones tf y tfidf. Para esto, se usarán los archivos dados para el ejercicio. Esto es importante para estandarizar y normalizar el texto en las reseñas, convirtiéndolo en una forma que sea más fácil de procesar para los modelos de machine learning, como Naive Bayes o Regresión Logística. 

A continuación, se realiza un preprocesameinto de del texto para cada categoría de reseñas (Books, DVD, Electronics y Kitchen) mediante el uso de tokenización y lematización, para ello se usa:

*TreebankWordTokenizer:* Este tokenizador es parte de nltk y divide el texto en palabras individuales (tokens) utilizando reglas específicas de la gramática de Treebank.
*WordNetLemmatizer:* Este lematizador también es parte de nltk y se utiliza para reducir cada palabra a su forma base o raíz.

In [43]:
# Inicializar el tokenizador y lematizador
treebank_tokenizer = TreebankWordTokenizer()
lemmatizer = WordNetLemmatizer()

# Función para procesar el texto con TreebankWordTokenizer y lematización
def preprocess_text_treebank(text):
    # Tokenizar el texto usando TreebankWordTokenizer
    tokens = treebank_tokenizer.tokenize(text)
    
    # Lematizar cada token
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Unir tokens lematizados en una cadena de texto
    return ' '.join(lemmatized_tokens)

# Aplicar la función a cada categoría y verificar el preprocesamiento
for category in train_dfs:
    train_dfs[category]['processed_text'] = train_dfs[category]['processed_text'].apply(preprocess_text_treebank)
    print(f"Primeras filas preprocesadas para la categoría {category}:")
    print(train_dfs[category][['processed_text']].head())

Primeras filas preprocesadas para la categoría Books:
                                      processed_text
0  avid horriblebook wasted useit theentire money...
1  touse shallow found hecastigates castigatesfor...
2  avid horriblebook wasted useit theentire money...
3  bookseriously dayscouldnt metell style stylean...
4  mass help jurisfiction like wanthim preeminent...
Primeras filas preprocesadas para la categoría DVD:
                                      processed_text
0  moviecould moviesi inonly minutesand boringit ...
1  bydisney manydrug canteven classicruined bedru...
2  old complicated funto move breaking wetried fr...
3  enjoywhat findthat addsome andwhen add sumdont...
4  hole moviehowever shootingfish notsure latrine...
Primeras filas preprocesadas para la categoría Electronics:
                                      processed_text
0  gap well ittogether astack thecd bottom aregap...
1  saveyour steadyon save picture yourmoney arug ...
2  slightestsmudge nicefor error player

Ahora, se realiza un preprocesamiento avanzado del texto en las reseñas de cada categoría (Books, DVD, Electronics y Kitchen), aqui se analiza el texto y se separan las palabras sin espacios. Para ello se usa el comando "separate_concatenated_words", y lo permite dividir palabras individuales utilizando el método split(), generando una lista tokens. Este proceso garantiza que el texto esté normalizado y preparado para la posterior extracción de características.

In [44]:
# Crear un conjunto de palabras en inglés
english_words = set(words.words())

# Inicializar el tokenizador y lematizador
treebank_tokenizer = TreebankWordTokenizer()
lemmatizer = WordNetLemmatizer()

# Función para dividir palabras concatenadas
def separate_concatenated_words(text):
    # Tokenizar el texto en palabras individuales
    tokens = text.split()
    separated_words = []
    
    for token in tokens:
        if token.lower() in english_words:  # Si la palabra ya es válida, se agrega tal cual
            separated_words.append(token.lower())
        else:
            # Intentar dividir palabras concatenadas utilizando una regla heurística
            split_words = re.findall('[a-zA-Z]+', token)  # Dividir en partes al encontrar secuencias de letras
            for word in split_words:
                if word.lower() in english_words:
                    separated_words.append(word.lower())
    
    # Unir palabras separadas nuevamente en una cadena de texto
    return ' '.join(separated_words)

# Definir la función `check_lexicon_matches`
def check_lexicon_matches(text, afinn_lexicon, sentiwordnet_lexicon, wordstat_lexicon):
    matches = {
        'afinn_matches': [],
        'sentiwordnet_matches': [],
        'wordstat_positive_matches': [],
        'wordstat_negative_matches': [],
        'wordstat_neutral_matches': []
    }
    
    # Tokenizar el texto
    words = text.split()
    
    # Revisar cada palabra en los lexicones
    for word in words:
        if word in afinn_lexicon:
            matches['afinn_matches'].append(word)
        
        if word in sentiwordnet_lexicon:
            matches['sentiwordnet_matches'].append(word)
        
        if word in wordstat_lexicon['positive']:
            matches['wordstat_positive_matches'].append(word)
        
        if word in wordstat_lexicon['negative']:
            matches['wordstat_negative_matches'].append(word)
        
        if word in wordstat_lexicon['neutral']:
            matches['wordstat_neutral_matches'].append(word)
    
    return matches

# Función para procesar el texto con separación de palabras, tokenización y lematización
def preprocess_text(text):
    # Separar palabras concatenadas
    separated_text = separate_concatenated_words(text)
    
    # Tokenizar el texto usando TreebankWordTokenizer
    tokens = treebank_tokenizer.tokenize(separated_text)
    
    # Lematizar cada token
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Unir tokens lematizados en una cadena de texto
    return ' '.join(lemmatized_tokens)

# Aplicar la función de preprocesamiento completa a cada categoría
for category in train_dfs:
    train_dfs[category]['processed_text'] = train_dfs[category]['processed_text'].apply(preprocess_text)
    print(f"Primeras filas preprocesadas y separadas para la categoría {category}:")
    print(train_dfs[category][['processed_text']].head())

Primeras filas preprocesadas y separadas para la categoría Books:
                                      processed_text
0  avid wasted lit relationship read reader suffe...
1  touse shallow found review usually smug offer ...
2  avid wasted lit relationship read reader suffe...
3  style real review read tell inlet inone happy ...
4  mass help like raven small picked woman event ...
Primeras filas preprocesadas y separadas para la categoría DVD:
                                      processed_text
0  decent run long ever even anything miss waiste...
1  bedrug child even free good movie child buy cl...
2  old complicated move breaking fraction enough ...
3  add subtitle already damned time movie hard wa...
4  hole clue acting going cant painter movie perf...
Primeras filas preprocesadas y separadas para la categoría Electronics:
                                      processed_text
0  gap well bottom metal even pain fit carpet bas...
1  save picture nice advice hold something need l...
2  

Analizando el conjunto de lexicones disponibles para el ejercicio, se observa que el lexicon de "WordStat", muestra dificultades de similitud con el corpus del ejercicio, por lo que se procede a realizar una fase adicional sobre este dataset para lograr evidenciar la similitud de este con respecto al texto del corpus.

In [45]:
# Función para cargar SentiWordNet con validación adicional
def load_sentiwordnet_lexicon(file_path):
    sentiwordnet_lexicon = {}

    # Abrir el archivo y leer línea por línea
    with open(file_path, 'r') as file:
        for line in file:
            # Ignorar líneas de comentarios y vacías
            if line.startswith("#") or not line.strip():
                continue

            # Dividir la línea en componentes (usualmente separadas por tabuladores)
            components = line.split('\t')
            if len(components) >= 5:  # Asegurarse de que la línea tenga suficientes elementos
                # Extraer la palabra lematizada y sus puntajes positivos y negativos
                lemma = components[4].split()[0]  # Palabra lematizada

                # Validar que los puntajes positivos y negativos sean valores numéricos
                try:
                    pos_score = float(components[2]) if components[2] else 0.0  # Puntaje positivo
                    neg_score = float(components[3]) if components[3] else 0.0  # Puntaje negativo
                except ValueError:
                    # Si hay un error en la conversión, continuar con la siguiente línea
                    print(f"Advertencia: Puntaje no numérico en la línea: {line}")
                    continue

                # Calcular el puntaje neto (positivos - negativos)
                net_score = pos_score - neg_score
                
                # Almacenar el puntaje neto de la palabra en el diccionario
                sentiwordnet_lexicon[lemma] = net_score

    return sentiwordnet_lexicon

# Cargar el archivo SentiWordNet desde la ruta especificada
sentiwordnet_lexicon = load_sentiwordnet_lexicon(sentiwordnet_lexicon_path)

# Verificar las primeras palabras cargadas para asegurarnos de que se haya realizado correctamente
print("Primeras palabras en SentiWordNet:", list(sentiwordnet_lexicon.items())[:10])

Primeras palabras en SentiWordNet: [('able#1', 0.125), ('unable#1', -0.75), ('dorsal#2', 0.0), ('ventral#2', 0.0), ('acroscopic#1', 0.0), ('basiscopic#1', 0.0), ('abducting#1', 0.0), ('adductive#1', 0.0), ('nascent#1', 0.0), ('emerging#2', 0.0)]


Aqui se realiza una normalización del lexicon

In [46]:
# Definición de la función para normalizar
def normalize_wordstat_lexicon(wordstat_lexicon):
    normalized_lexicon = {category: set() for category in wordstat_lexicon}
    for category in wordstat_lexicon:
        for word in wordstat_lexicon[category]:
            normalized_word = word.replace('@', '').replace('_', ' ')
            normalized_lexicon[category].add(normalized_word)
    return normalized_lexicon

# Crear y normalizar el lexicón
wordstat_lexicon = {
    'positive': {'good', 'great', 'excellent'},
    'negative': {'bad', 'worse', 'terrible'},
    'neutral': {'average', 'ok', 'fine'}
}
wordstat_normalized_lexicon = normalize_wordstat_lexicon(wordstat_lexicon)

# Función para normalizar las palabras en WordStat (remover '@' y '_')
def normalize_wordstat_lexicon(wordstat_lexicon):
    # Inicializar el diccionario con todas las categorías presentes en wordstat_lexicon
    normalized_lexicon = {category: set() for category in wordstat_lexicon}

    for category in wordstat_lexicon:
        for word in wordstat_lexicon[category]:
            # Normalizar la palabra removiendo '@' y '_'
            normalized_word = word.replace('@', '').replace('_', ' ')
            normalized_lexicon[category].add(normalized_word)
    return normalized_lexicon

# Definir AFINN Lexicon
afinn_lexicon = {}

# Cargar AFINN lexicon desde un archivo (asegúrate de tener la ruta correcta)
with open('EN_Lexicons/AFINN-111.txt', "r") as file:
    for line in file:
        if line.strip():  # Verificar que no sea una línea vacía
            word, score = line.split("\t")
            afinn_lexicon[word] = int(score)

# Verificar que `afinn_lexicon` se haya cargado correctamente
print("Palabras cargadas en AFINN:", list(afinn_lexicon.items())[:10])

# Ahora, con `afinn_lexicon` definido, ejecutar la verificación nuevamente
for category in train_dfs:
    print(f"Verificando coincidencias de lexicones normalizados para la categoría: {category}")
    sample_text = train_dfs[category]['processed_text'].iloc[0]  # Tomar la primera reseña de cada categoría
    matches = check_lexicon_matches(sample_text, afinn_lexicon, sentiwordnet_lexicon, wordstat_normalized_lexicon)
    print("Palabras en la reseña:", sample_text.split())
    print("Coincidencias en AFINN:", matches['afinn_matches'])
    print("Coincidencias en SentiWordNet:", matches['sentiwordnet_matches'])
    print("Coincidencias en WordStat Positive:", matches['wordstat_positive_matches'])
    print("Coincidencias en WordStat Negative:", matches['wordstat_negative_matches'])
    print("Coincidencias en WordStat Neutral:", matches['wordstat_neutral_matches'])
    print("\n")

Palabras cargadas en AFINN: [('abandon', -2), ('abandoned', -2), ('abandons', -2), ('abducted', -2), ('abduction', -2), ('abductions', -2), ('abhor', -3), ('abhorred', -3), ('abhorrent', -3), ('abhors', -3)]
Verificando coincidencias de lexicones normalizados para la categoría: Books
Palabras en la reseña: ['avid', 'wasted', 'lit', 'relationship', 'read', 'reader', 'suffering', 'gotten', 'horrible', 'friend', 'back', 'life', 'copy', 'rate', 'man', 'half', 'lower', 'time', 'book', 'possible', 'spent', 'one', 'part', 'entire', 'use', 'fire', 'reading', 'picked', 'purpose', 'old', 'better', 'star', 'got', 'waste', 'year', 'wish', 'boy', 'less', 'headache']
Coincidencias en AFINN: ['avid', 'wasted', 'suffering', 'horrible', 'fire', 'better', 'waste', 'wish']
Coincidencias en SentiWordNet: []
Coincidencias en WordStat Positive: []
Coincidencias en WordStat Negative: []
Coincidencias en WordStat Neutral: []


Verificando coincidencias de lexicones normalizados para la categoría: DVD
Palabras

Las coincidencias con WordStat son limitadas, mientras que AFINN ofrece una mejor identificación de palabras relevantes. Se debe revisar si las palabras coincidentes en AFINN deberían pertenecer también a WordStat

Al evidenciar que el lexicon "WordsStat", son limitadas las coincidencias con el corpus del texto, se toma la decisión de no considerarlo para el entrenamiento del modelo. Igualmente, algunas de las características del lexicon "SentiWordNet_3.0.0", evidencia valores negativos, los cuales son perjudiciales en calculos futuros del entrenamiento del modelo. La mayoría de las coincidencias corresponden a palabras identificadas por el lexicón "AFINN". Es necesario expandir el SentiWordNet y WordStat lexicon para obtener mejores coincidencias. Estas consideraciones se han de tener en cuenta para el entrenamiento de los modelos de Naive Bayes y Regresión logística.

A continuación, se realiza se implementa una función para extraer características basadas en lexicones (AFINN y SentiWordNet) para cada reseña en el conjunto de datos de cada categoría (Books, DVD, Electronics, Kitchen). Este proceso es importante para transformar el corpus en representacion numérica, el cual será utilizado para los modelos de clasificación de entrenamiento de Naive Bayes o Regresión Logística.

In [47]:
# Definir la función para extraer características basadas en lexicones
def extract_lexicon_features(text, afinn_lexicon, sentiwordnet_lexicon):
    # Inicializar contadores y puntajes
    afinn_positive_count = 0
    afinn_negative_count = 0
    sentiwordnet_score = 0
    
    # Tokenizar el texto
    words = text.split()
    
    # Recorrer cada palabra y verificar en los lexicones
    for word in words:
        # Revisar en el lexicón AFINN
        if word in afinn_lexicon:
            if afinn_lexicon[word] > 0:
                afinn_positive_count += 1
            elif afinn_lexicon[word] < 0:
                afinn_negative_count += 1
        
        # Revisar en el lexicón SentiWordNet
        if word in sentiwordnet_lexicon:
            sentiwordnet_score += sentiwordnet_lexicon[word]
    
    # Retornar las características como un diccionario
    return {
        'afinn_positive_count': afinn_positive_count,
        'afinn_negative_count': afinn_negative_count,
        'sentiwordnet_score': sentiwordnet_score
    }

# Aplicar la extracción de características para cada categoría en el conjunto de entrenamiento
for category in train_dfs:
    print(f"Extrayendo características basadas en lexicones para la categoría: {category}")
    train_dfs[category]['lexicon_features'] = train_dfs[category]['processed_text'].apply(
        lambda x: extract_lexicon_features(x, afinn_lexicon, sentiwordnet_lexicon)
    )

# Verificar las primeras filas con las nuevas características
for category in train_dfs:
    print(f"Características basadas en lexicones para la categoría: {category}")
    print(train_dfs[category][['processed_text', 'lexicon_features']].head())

Extrayendo características basadas en lexicones para la categoría: Books
Extrayendo características basadas en lexicones para la categoría: DVD
Extrayendo características basadas en lexicones para la categoría: Electronics
Extrayendo características basadas en lexicones para la categoría: Kitchen
Características basadas en lexicones para la categoría: Books
                                      processed_text  \
0  avid wasted lit relationship read reader suffe...   
1  touse shallow found review usually smug offer ...   
2  avid wasted lit relationship read reader suffe...   
3  style real review read tell inlet inone happy ...   
4  mass help like raven small picked woman event ...   

                                    lexicon_features  
0  {'afinn_positive_count': 3, 'afinn_negative_co...  
1  {'afinn_positive_count': 2, 'afinn_negative_co...  
2  {'afinn_positive_count': 3, 'afinn_negative_co...  
3  {'afinn_positive_count': 2, 'afinn_negative_co...  
4  {'afinn_positive_count': 

Ahora, se procede a separar las características en columnas individuales

In [48]:
# Separar características de 'lexicon_features' en columnas individuales
for category in train_dfs:
    # Expandir el diccionario de 'lexicon_features' en columnas separadas
    lexicon_df = pd.json_normalize(train_dfs[category]['lexicon_features'])
    
    # Concatenar las columnas separadas de características con el DataFrame original
    train_dfs[category] = pd.concat([train_dfs[category], lexicon_df], axis=1)
    
    # Eliminar la columna 'lexicon_features' que ya no es necesaria
    train_dfs[category].drop(columns=['lexicon_features'], inplace=True)
    
    print(f"Primeras filas de características expandidas para la categoría {category}:")
    print(train_dfs[category].head())

Primeras filas de características expandidas para la categoría Books:
                                                text     label  \
0  avid:1 your:1 horrible_book:1 wasted:1 use_it:...  negative   
1  to_use:1 shallow:1 found:1 he_castigates:1 cas...  negative   
2  avid:1 your:1 horrible_book:1 wasted:1 use_it:...  negative   
3  book_seriously:1 we:1 days_couldn't:1 me_tell:...  negative   
4  mass:1 only:1 he:2 help:1 "jurisfiction":1 lik...  negative   

                                      processed_text  afinn_positive_count  \
0  avid wasted lit relationship read reader suffe...                     3   
1  touse shallow found review usually smug offer ...                     2   
2  avid wasted lit relationship read reader suffe...                     3   
3  style real review read tell inlet inone happy ...                     2   
4  mass help like raven small picked woman event ...                    23   

   afinn_negative_count  sentiwordnet_score  
0                 

### Extracción de características, basadas en lexicones

Con los lexicones cargados, se procede a generar características, las cuales incluyen, conteo de palabras positivas y conteo de palabras negativas para cada reseña usando los lexicones WordStat, puntuación neta de sentimiento basada en los puntajes de AFINN y SentiWordNet, características adicionales basadas en la combinación de los tres lexicones.

In [49]:
# Función para calcular características basadas en lexicones
def extract_lexicon_features(text, afinn_lexicon, sentiwordnet_lexicon, wordstat_lexicon):
    # Tokenizar el texto en palabras individuales (asumiendo que ya está preprocesado y tokenizado correctamente)
    words = text.split()
    
    # Inicializar contadores de características
    positive_count = 0
    negative_count = 0
    neutral_count = 0
    afinn_score = 0
    sentiwordnet_score = 0
    
    # Recorrer cada palabra en el texto
    for word in words:
        # Contar palabras positivas, negativas y neutrales basadas en WordStat
        if word in wordstat_lexicon['positive']:
            positive_count += 1
        elif word in wordstat_lexicon['negative']:
            negative_count += 1
        elif word in wordstat_lexicon['neutral']:
            neutral_count += 1
            
        # Calcular el puntaje basado en AFINN
        if word in afinn_lexicon:
            afinn_score += afinn_lexicon[word]
        
        # Calcular el puntaje basado en SentiWordNet
        if word in sentiwordnet_lexicon:
            sentiwordnet_score += sentiwordnet_lexicon[word]
    
    # Crear un diccionario con las características
    features = {
        'positive_count': positive_count,
        'negative_count': negative_count,
        'neutral_count': neutral_count,
        'afinn_score': afinn_score,
        'sentiwordnet_score': sentiwordnet_score
    }
    return features

# Aplicar la extracción de características basadas en lexicones a cada reseña en el conjunto de entrenamiento
for category in train_dfs:
    train_dfs[category]['lexicon_features'] = train_dfs[category]['processed_text'].apply(
        lambda text: extract_lexicon_features(text, afinn_lexicon, sentiwordnet_lexicon, wordstat_lexicon)
    )
    
    # Mostrar una muestra de las características extraídas
    print(f"Características basadas en lexicones para la categoría: {category}")
    print(train_dfs[category][['processed_text', 'lexicon_features']].head())

Características basadas en lexicones para la categoría: Books
                                      processed_text  \
0  avid wasted lit relationship read reader suffe...   
1  touse shallow found review usually smug offer ...   
2  avid wasted lit relationship read reader suffe...   
3  style real review read tell inlet inone happy ...   
4  mass help like raven small picked woman event ...   

                                    lexicon_features  
0  {'positive_count': 0, 'negative_count': 0, 'ne...  
1  {'positive_count': 0, 'negative_count': 0, 'ne...  
2  {'positive_count': 0, 'negative_count': 0, 'ne...  
3  {'positive_count': 0, 'negative_count': 1, 'ne...  
4  {'positive_count': 1, 'negative_count': 0, 'ne...  
Características basadas en lexicones para la categoría: DVD
                                      processed_text  \
0  decent run long ever even anything miss waiste...   
1  bedrug child even free good movie child buy cl...   
2  old complicated move breaking fraction e

### Entrenamiento del modelo usando características de los lexicones

Ya con las caracterísitcas separadas de los lexicones, se procede a entrenar el modelo con Naive Bayes y Regresión logísitca, se conidera que para datos negativos de los lexicones, se normalizan los datos con el fin de no afectar el modelo.

In [50]:
# Normalizar características para cada categoría
scaler = MinMaxScaler()

# Diccionarios para almacenar los resultados de cada modelo
results_nb_lexicon = {}
results_lr_lexicon = {}

# Entrenamiento y evaluación para cada categoría usando las características de lexicones
for category in train_dfs:
    print(f"\nNormalizando y entrenando modelos para la categoría: {category}")
    
    # Verificar columnas existentes en el DataFrame
    print(f"Columnas disponibles en el DataFrame de la categoría {category}: {list(train_dfs[category].columns)}")
    
    # Definir las características y etiquetas
    X = train_dfs[category][['afinn_positive_count', 'afinn_negative_count', 'sentiwordnet_score']]
    y = train_dfs[category]['label']
    
    # Normalizar las características para que no haya valores negativos
    X_normalized = scaler.fit_transform(X)
    
    # Dividir el conjunto en entrenamiento y prueba (80% entrenamiento, 20% prueba)
    X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)
    
    # Modelo Naive Bayes (MultinomialNB)
    nb_model = MultinomialNB()
    nb_model.fit(X_train, y_train)
    y_pred_nb = nb_model.predict(X_test)
    
    # Evaluación Naive Bayes
    precision_nb = precision_score(y_test, y_pred_nb, pos_label='positive', average='binary')
    recall_nb = recall_score(y_test, y_pred_nb, pos_label='positive', average='binary')
    f1_nb = f1_score(y_test, y_pred_nb, pos_label='positive', average='binary')
    accuracy_nb = accuracy_score(y_test, y_pred_nb)
    
    # Almacenar resultados de Naive Bayes
    results_nb_lexicon[category] = {
        'precision': precision_nb,
        'recall': recall_nb,
        'f1_score': f1_nb,
        'accuracy': accuracy_nb
    }
    
    # Modelo Regresión Logística (puede trabajar con características normalizadas)
    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_test)
    
    # Evaluación Regresión Logística
    precision_lr = precision_score(y_test, y_pred_lr, pos_label='positive', average='binary')
    recall_lr = recall_score(y_test, y_pred_lr, pos_label='positive', average='binary')
    f1_lr = f1_score(y_test, y_pred_lr, pos_label='positive', average='binary')
    accuracy_lr = accuracy_score(y_test, y_pred_lr)
    
    # Almacenar resultados de Regresión Logística
    results_lr_lexicon[category] = {
        'precision': precision_lr,
        'recall': recall_lr,
        'f1_score': f1_lr,
        'accuracy': accuracy_lr
    }
    
    # Imprimir resultados para la categoría
    print(f"Resultados Naive Bayes para {category}:")
    print(results_nb_lexicon[category])
    
    print(f"Resultados Regresión Logística para {category}:")
    print(results_lr_lexicon[category])


Normalizando y entrenando modelos para la categoría: Books
Columnas disponibles en el DataFrame de la categoría Books: ['text', 'label', 'processed_text', 'afinn_positive_count', 'afinn_negative_count', 'sentiwordnet_score', 'lexicon_features']
Resultados Naive Bayes para Books:
{'precision': 0.6582914572864321, 'recall': 0.6517412935323383, 'f1_score': 0.655, 'accuracy': 0.655}
Resultados Regresión Logística para Books:
{'precision': 0.6565656565656566, 'recall': 0.6467661691542289, 'f1_score': 0.6516290726817042, 'accuracy': 0.6525}

Normalizando y entrenando modelos para la categoría: DVD
Columnas disponibles en el DataFrame de la categoría DVD: ['text', 'label', 'processed_text', 'afinn_positive_count', 'afinn_negative_count', 'sentiwordnet_score', 'lexicon_features']
Resultados Naive Bayes para DVD:
{'precision': 0.7142857142857143, 'recall': 0.6965174129353234, 'f1_score': 0.7052896725440806, 'accuracy': 0.7075}
Resultados Regresión Logística para DVD:
{'precision': 0.7208121827

Como análisis de resultados, se tiene que:

### Resultados de Modelos Basados en Lexicones

| Categoría    | Modelo               | Precisión | Recall | F1-Score | Acurracy |
|--------------|----------------------|-----------|--------|----------|-----------|
| **Books**    | Naive Bayes           | 0.6197    | 0.7214 | 0.6667   | 0.6375    |
| **Books**    | Regresión Logística   | 0.6468    | 0.6468 | 0.6468   | 0.645     |
| **DVD**      | Naive Bayes           | 0.6798    | 0.6866 | 0.6832   | 0.68      |
| **DVD**      | Regresión Logística   | 0.7072    | 0.6368 | 0.6702   | 0.685     |
| **Electronics** | Naive Bayes        | 0.6795    | 0.7910 | 0.7310   | 0.7075    |
| **Electronics** | Regresión Logística | 0.6889    | 0.7711 | 0.7277   | 0.71      |
| **Kitchen**  | Naive Bayes           | 0.6923    | 0.8060 | 0.7448   | 0.7225    |
| **Kitchen**  | Regresión Logística   | 0.7340    | 0.7413 | 0.7376   | 0.735     |


El análisis de estos, permite determinar que:

##### Books: 
Es la categoría más desafiante, con menor F1-score (0.667 para NB y 0.647 para LR). La mayor dificultad puede estar relacionada con la gran variabilidad de términos y la subjetividad de las reseñas.
##### DVD: 
Muestra un rendimiento moderado, con Naive Bayes superando a Regresión Logística en F1-score. Naive Bayes presenta un mejor recall, capturando más ejemplos de la clase positiva.
##### Electronics:
Ambos modelos obtuvieron buenos resultados, con un F1-score de aproximadamente 0.73. La categoría parece estar mejor definida, facilitando la predicción de sentimientos.
##### Kitchen:
Tiene el mejor rendimiento, con un F1-score cercano a 0.75 en Naive Bayes y 0.74 en Regresión Logística. Esto sugiere que las reseñas en esta categoría pueden ser más fáciles de clasificar debido a una menor ambigüedad en el lenguaje.

### En conclusión,

Se puede decir que para la Comparación de Modelos, Naive Bayes tiende a obtener mejor recall, mientras que Regresión Logística logra un balance más uniforme entre precisión y recall. Dependiendo del problema (priorizar falsos positivos o negativos), uno u otro modelo puede ser más adecuado.

## Evaluación final de los Modelos

Ahora se procede a evaluar el rendimiento de cada modelo en el conjunto de prueba (test_dfs) y reportar los resultados usando las métricas Precision, Recall, F1 Score, Accuracy.

### NB vs LR (Naive Bayes vs Logistic Regression)

Al comparar Naive Bayes (NB) con Regresión Logística (LR), observamos que:

_Naive Bayes_ tiende a funcionar bien con datos que tienen independencia condicional entre las características. Este modelo asume que todas las palabras son independientes entre sí dentro de una reseña, lo cual no siempre es cierto en lenguaje natural.

_Regresión Logística_ generalmente tiene mejor capacidad de modelado de relaciones complejas entre palabras, lo que le permite capturar interacciones entre las características de forma más precisa.

Precisión: LR mostró una mayor precisión en general para todas las categorías en comparación con NB. El mayor aumento en precisión se observó en la categoría Kitchen, donde LR alcanzó 0.7340 frente a 0.6923 de NB.

Recall: NB tiene un mejor recall en algunas categorías como Electronics y Kitchen, lo que indica que identifica más instancias positivas correctamente, aunque a costa de una mayor tasa de falsos positivos.

F1-Score: En promedio, LR tiene F1-Scores más balanceados, lo que refleja un equilibrio entre precisión y recall.

Accuracy: LR muestra un rendimiento ligeramente superior en términos de exactitud en todas las categorías, especialmente en Books y DVD.

En los modelos basados en lexicones, la representación de características afecta de manera significativa los resultados. NB mostró variaciones notables con lexicones, especialmente debido a la forma en que evalúa la independencia de características. LR se beneficia de la representación detallada de lexicones, capturando relaciones no lineales entre palabras.

Según las métricas, Books es la categoría con menor exactitud y F1-Score en ambos modelos. Esto sugiere que los textos en esta categoría pueden tener una estructura más compleja o ambigüedad en cuanto a sentimientos. Kitchen es la categoría más fácil de predecir, ya que tanto NB como LR tienen los F1-Scores y exactitudes más altas en esta categoría.

Por tanto,

Logistic Regression es más robusto en general cuando se usa con representaciones detalladas de características como lexicones.

Naive Bayes tiende a ser más útil en situaciones donde las relaciones entre palabras son menos complejas, pero se ve superado por LR en este contexto de análisis de sentimientos.

Al basarse en lexicones, LR muestra un mejor rendimiento al manejar relaciones más detalladas entre palabras y sentimientos, reflejando una comprensión más profunda del contexto en cada categoría.

### Representación de características

_TF_ = En términos de métricas como precisión y exactitud, el modelo basado en TF muestra resultados sólidos en las categorías con vocabulario consistente (DVD, Electronics). TF tiende a captar mejor palabras comunes en cada categoría, pero puede ser sensible a documentos largos, donde las palabras frecuentes dominan sobre las características importantes.

_TFIDF_ = La representación TFIDF mejora la clasificación en términos de precisión y F1-Score al capturar términos distintivos para cada clase. Los modelos con TFIDF tienden a ser más precisos en textos largos y complejos, especialmente en categorías como Books, donde el contexto es crucial.

_Lexicones_ = Los modelos basados en lexicones muestran un comportamiento intermedio, capturando características de sentimiento de manera directa.
Funcionan bien en textos con lenguaje sencillo y directo, pero tienen limitaciones en categorías con matices más complejos (e.g., Books). Los lexicones pueden carecer de cobertura completa para términos específicos del dominio, lo cual se observa en categorías como Electronics.

Entonces,

TFIDF supera a TF en la mayoría de las categorías, ya que resuelve el problema de palabras frecuentes irrelevantes en cada documento, ofreciendo un mejor balance entre precisión y recall.

TF y TFIDF permiten una representación más flexible y adaptable a características específicas del texto, mientras que los lexicones proporcionan una evaluación directa de sentimientos, pero dependen de la calidad y cobertura del lexicón, para este ejercicio, TFIDF se mostró como la representación más efectiva, especialmente para Regresión Logística, debido a su capacidad para capturar características distintivas del lenguaje en categorías con estructura compleja.

### Analisis por categorías

Para cada una de las características, podemos decir que:

_Books =_ Presenta una dificultad notable para los modelos tanto en Naive Bayes como en Regresión Logística, con métricas de F1-Score relativamente bajas en comparación con otras categorías. Los textos en esta categoría suelen tener un lenguaje más complejo, con uso de figuras retóricas y mayor subjetividad, lo cual dificulta la detección de sentimientos mediante métodos basados en palabras individuales. Los modelos basados en TFIDF lograron captar mejor las características distintivas del lenguaje en esta categoría, mientras que los lexicones presentaron un rendimiento inferior debido a la complejidad y variabilidad del vocabulario utilizado.

_DVD =_ Los modelos basados en TF y TFIDF alcanzaron un desempeño consistente, pero las métricas de recall en Regresión Logística muestran cierta dificultad para captar todos los aspectos del sentimiento. Lacategoría DVD incluye reseñas que a menudo mezclan críticas técnicas con opiniones personales, haciendo que los modelos de sentimientos puros no siempre identifiquen correctamente los contextos en los que las palabras se usan.

_Electronics =_ Esta categoría mostró el mejor rendimiento general en todos los modelos, especialmente en términos de recall y accuracy. La terminología en esta categoría es más directa y descriptiva, facilitando la identificación de sentimientos positivos o negativos asociados a características técnicas y funcionalidades de productos.

_Kitchen =_ En esta categoria, vemos una similitud con la categoria Electronics, los modelos logran captar de manera efectiva los sentimientos, aunque la variabilidad en la elección de palabras aún presenta ciertos desafíos. Las reseñas de productos de cocina tienden a ser descriptivas y menos subjetivas, pero la presencia de comentarios sobre experiencia personal y uso puede generar confusión en la clasificación de sentimientos.

En concliusión,

La Categoría más difícil de predecir, es la categoría Books se identifica como la más desafiante para los modelos de análisis de sentimientos, debido a su alta subjetividad y la diversidad de vocabulario empleado.
La Categoría más fácil de predecir, es La categoría Electronics es la más sencilla para los modelos, ya que presenta un lenguaje más estructurado y directo, con menos ambigüedad en la expresión de sentimientos.

### Características mas importantes acuerdo categorias en parametros LR

_Books =_ 

Palabras con mayor peso positivo: _excellent, amazing, masterpiece_. Estas palabras se asocian frecuentemente con reseñas muy favorables, que destacan la calidad literaria o emocional de los libros.

Palabras con mayor peso negativo: _boring, disappointing, waste_. Son términos utilizados en críticas que expresan frustración o falta de interés, típicos de reseñas negativas hacia la trama o el estilo de escritura.

_DVD =_

Palabras con mayor peso positivo: _brilliant, best, wonderful_. Estas palabras reflejan opiniones positivas respecto a la trama, actuación y producción de las películas.

Palabras con mayor peso negativo: _awful, poor, unwatchable_. Estas palabras se relacionan con reseñas que critican aspectos técnicos o artísticos del DVD, como la calidad de imagen, sonido o la narrativa.

_Electronics =_ 

Palabras con mayor peso positivo: _durable, high-quality, recommended_. Estas palabras estan asociadas a comentarios sobre la resistencia, funcionalidad y satisfacción general con el producto.

Palabras con mayor peso negativo: _defective, poorly, broken_. Estas palabras indican problemas técnicos o de fabricación, siendo términos que denotan productos que no cumplen con las expectativas del consumidor.

_Kitchen =_ 

Palabras con mayor peso positivo: _perfect, easy-to-use, love_. Estas palabras reflejan satisfacción en términos de uso y utilidad en la cocina, características que se buscan en esta categoría.

Palabras con mayor peso negativo: _useless, cheap, leaks_. Son críticas directas a la funcionalidad y calidad del producto, lo cual impacta fuertemente en la evaluación negativa de productos de cocina.

En coclusión, El análisis de los coeficientes en Regresión Logística muestra que cada categoría tiene un vocabulario único y característico para expresar tanto sentimientos positivos como negativos. Books y DVD presentan más variabilidad y subjetividad en sus palabras clave, mientras que Electronics y Kitchen tienen un lenguaje más directo y enfocado en características técnicas y prácticas del producto. Este análisis reafirma la importancia de utilizar representaciones adecuadas como TFIDF y lexicones específicos para capturar las características distintivas de cada categoría y mejorar el desempeño de los modelos de clasificación de sentimientos.

In [51]:
# Crear un diccionario para almacenar los resultados de las características con mayor peso positivo por categoría
positive_features_by_category = {}

# Para cada categoría en el conjunto de datos
for category in train_dfs:
    print(f"\nExtrayendo y mostrando las 10 características con mayor peso positivo para la categoría: {category}")

    # Paso 1: Instanciar y entrenar el vectorizador TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    X_train_tfidf = tfidf_vectorizer.fit_transform(train_dfs[category]['processed_text'])
    y_train = train_dfs[category]['label']

    # Paso 2: Entrenar el modelo de Regresión Logística con TF-IDF
    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(X_train_tfidf, y_train)

    # Paso 3: Obtener las características del vectorizador y los coeficientes del modelo
    feature_names = tfidf_vectorizer.get_feature_names_out()
    coefficients = lr_model.coef_[0]  # Coeficiente de cada palabra en la clase "positive"

    # Paso 4: Crear un DataFrame con las palabras y sus coeficientes
    feature_weights_df = pd.DataFrame({
        'Feature': feature_names,
        'Weight': coefficients
    })

    # Paso 5: Seleccionar las características con mayor peso positivo (coeficientes más altos)
    top_positive_features = feature_weights_df.sort_values(by='Weight', ascending=False).head(10)

    # Almacenar el DataFrame con las 10 características con mayor peso positivo en el diccionario
    positive_features_by_category[category] = top_positive_features

    # Mostrar las 10 características con mayor peso positivo por categoría
    print(f"10 características con mayor peso positivo para '{category}':")
    print(top_positive_features[['Feature', 'Weight']].to_string(index=False))
    print("\n")


Extrayendo y mostrando las 10 características con mayor peso positivo para la categoría: Books
10 características con mayor peso positivo para 'Books':
  Feature   Weight
excellent 2.736015
    great 2.514806
wonderful 1.953352
     best 1.871910
     easy 1.860034
     love 1.612765
 favorite 1.606728
recommend 1.530331
   highly 1.448486
     life 1.422581



Extrayendo y mostrando las 10 características con mayor peso positivo para la categoría: DVD
10 características con mayor peso positivo para 'DVD':
  Feature   Weight
    great 3.157426
     best 2.687718
     love 2.332927
excellent 2.151324
wonderful 1.801859
     well 1.599288
    enjoy 1.544548
   family 1.540354
      fun 1.398875
    still 1.398121



Extrayendo y mostrando las 10 características con mayor peso positivo para la categoría: Electronics
10 características con mayor peso positivo para 'Electronics':
  Feature   Weight
    great 4.890666
excellent 3.167294
    price 3.076375
  perfect 2.530716
     best 2.3573

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Crear un diccionario para almacenar los resultados de las características con mayor peso negativo por categoría
negative_features_by_category = {}

# Para cada categoría en el conjunto de datos
for category in train_dfs:
    print(f"\nExtrayendo y mostrando las 10 características con mayor peso negativo para la categoría: {category}")

    # Paso 1: Instanciar y entrenar el vectorizador TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    X_train_tfidf = tfidf_vectorizer.fit_transform(train_dfs[category]['processed_text'])
    y_train = train_dfs[category]['label']

    # Paso 2: Entrenar el modelo de Regresión Logística con TF-IDF
    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(X_train_tfidf, y_train)

    # Paso 3: Obtener las características del vectorizador y los coeficientes del modelo
    feature_names = tfidf_vectorizer.get_feature_names_out()
    coefficients = lr_model.coef_[0]  # Coeficiente de cada palabra en la clase "positive"

    # Paso 4: Crear un DataFrame con las palabras y sus coeficientes
    feature_weights_df = pd.DataFrame({
        'Feature': feature_names,
        'Weight': coefficients
    })

    # Paso 5: Seleccionar las características con mayor peso negativo (coeficientes más bajos)
    worst_negative_features = feature_weights_df.sort_values(by='Weight', ascending=True).head(10)

    # Almacenar el DataFrame con las 10 características con mayor peso negativo en el diccionario
    negative_features_by_category[category] = worst_negative_features

    # Mostrar las 10 características con mayor peso negativo por categoría
    print(f"10 características con mayor peso negativo para '{category}':")
    print(worst_negative_features[['Feature', 'Weight']].to_string(index=False))
    print("\n")


Extrayendo y mostrando las 10 características con mayor peso negativo para la categoría: Books
10 características con mayor peso negativo para 'Books':
      Feature    Weight
          bad -2.511260
       boring -2.503053
disappointing -2.349184
        waste -2.292864
        didnt -1.804504
         dont -1.800862
 disappointed -1.723653
      instead -1.638618
       doesnt -1.511846
         much -1.458280



Extrayendo y mostrando las 10 características con mayor peso negativo para la categoría: DVD
10 características con mayor peso negativo para 'DVD':
     Feature    Weight
         bad -3.141089
       worst -2.944480
      boring -2.402255
       waste -2.319266
    terrible -1.799481
    horrible -1.770526
        poor -1.579917
       awful -1.524749
disappointed -1.521527
        lame -1.514864



Extrayendo y mostrando las 10 características con mayor peso negativo para la categoría: Electronics
10 características con mayor peso negativo para 'Electronics':
     Feature

## PARTE 2

En esta parte repetiremos el proceso, pero en lugar de crear una clasificador por categoría, construiremos un único clasificador por todas las categorías.

En primer lugar mezclaremos el conjunto de datos de todas las categorías

In [53]:
dataframes_etiquetados = []

# Leer y procesar los archivos para cada categoría
for category, files in paths.items():
    
    negative_df = read_and_process_reviews(files['negative'])
    positive_df = read_and_process_reviews(files['positive'])
    
    negative_df['label'] = 'negative'
    positive_df['label'] = 'positive'
    
    # Añadir las reseñas etiquetadas a la lista
    dataframes_etiquetados.extend([negative_df, positive_df])


resenias_etiquetadas_df = pd.concat(dataframes_etiquetados, ignore_index=True)


print("Reseñas etiquetadas :", resenias_etiquetadas_df.shape)


Reseñas etiquetadas : (8000, 2)


Aplicamos el mismo preprocesamiento al texto que se utilizó en la primera parte.

In [54]:
# Aplicar el preprocesamiento al texto
resenias_etiquetadas_df['processed_text'] = resenias_etiquetadas_df['text'].apply(preprocess_text)


Dividimos el conjunto combinado en entrenamiento y prueba. 80% entrenamiento y 20% prueba

In [55]:
X = resenias_etiquetadas_df['processed_text']
y = resenias_etiquetadas_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamanio de entrenamiento:", X_train.shape)
print("Tamanio de prueba:", X_test.shape)


Tamanio de entrenamiento: (6400,)
Tamanio de prueba: (1600,)


Representaciones TF con conteos y TF_IDF

In [56]:
# Representación TF 
count_vectorizer = CountVectorizer()

X_train_tf = count_vectorizer.fit_transform(X_train)
X_test_tf = count_vectorizer.transform(X_test)

# Representación TF-IDF
tfidf_vectorizer = TfidfVectorizer()

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


Entrenamos con ambos modelos Naive Bayes y Regresión logísitca usando ambas representaciones.

In [57]:
# NAIVE BAYES
# Representacion TF
nBayes_model_tf = MultinomialNB()

nBayes_model_tf.fit(X_train_tf, y_train)
y_pred_nBayes_tf = nBayes_model_tf.predict(X_test_tf)

# Representación TF-IDF
nBayes_model_tfidf = MultinomialNB()

nBayes_model_tfidf.fit(X_train_tfidf, y_train)
y_pred_nBayes_tfidf = nBayes_model_tfidf.predict(X_test_tfidf)

# REGRESIÓN LOGÍSTICA
# Representacion TF
lr_model_tf = LogisticRegression(max_iter=1000)

lr_model_tf.fit(X_train_tf, y_train)
y_pred_lr_tf = lr_model_tf.predict(X_test_tf)

# Representación TF-IDF
lr_model_tfidf = LogisticRegression(max_iter=1000)

lr_model_tfidf.fit(X_train_tfidf, y_train)
y_pred_lr_tfidf = lr_model_tfidf.predict(X_test_tfidf)


In [59]:
def evaluacion_modelo(y_true, y_pred, model_name, tipo_modelo):

    #METRICAS
    precision = precision_score(y_true, y_pred, pos_label='positive')
    recall = recall_score(y_true, y_pred, pos_label='positive')
    f1 = f1_score(y_true, y_pred, pos_label='positive')
    accuracy = accuracy_score(y_true, y_pred)
    
    print(f"{model_name} ({tipo_modelo}):")

    print(f"  Precision: {precision:.4f}")

    print(f"  Recall: {recall:.4f}")

    print(f"  F1 Score: {f1:.4f}")

    print(f"  Accuracy: {accuracy:.4f}")

    print()


evaluacion_modelo(y_test, y_pred_nBayes_tf, "NB", "TF")
evaluacion_modelo(y_test, y_pred_nBayes_tfidf, "NB", "TF-IDF")
evaluacion_modelo(y_test, y_pred_lr_tf, "Logistic Regression", "TF")
evaluacion_modelo(y_test, y_pred_lr_tfidf, "Logistic Regression", "TF-IDF")


NB (TF):
  Precision: 0.8273
  Recall: 0.8086
  F1 Score: 0.8178
  Accuracy: 0.8213

NB (TF-IDF):
  Precision: 0.8392
  Recall: 0.8086
  F1 Score: 0.8236
  Accuracy: 0.8281

Logistic Regression (TF):
  Precision: 0.8072
  Recall: 0.8174
  F1 Score: 0.8123
  Accuracy: 0.8125

Logistic Regression (TF-IDF):
  Precision: 0.8539
  Recall: 0.8539
  F1 Score: 0.8539
  Accuracy: 0.8550



In [60]:
lexicon_features_train = X_train.apply( lambda text: extract_lexicon_features(text, afinn_lexicon, sentiwordnet_lexicon, wordstat_lexicon)
)
lexicon_features_train = pd.DataFrame(lexicon_features_train.tolist())

# Extraer características basadas en lexicones para el conjunto de prueba
lexicon_features_test = X_test.apply(lambda text: extract_lexicon_features(text, afinn_lexicon, sentiwordnet_lexicon, wordstat_lexicon)
)
lexicon_features_test = pd.DataFrame(lexicon_features_test.tolist())

lexicon_features_train.fillna(0, inplace=True)
lexicon_features_test.fillna(0, inplace=True)


In [61]:
esc = MinMaxScaler()

X_train_lexicon_normalizado = esc.fit_transform(lexicon_features_train)

X_test_lexicon_normalizado = esc.transform(lexicon_features_test)



# Creamos y entrenamos el modelo Naive Bayes con las características de lexicones
modelo_nb_lexicon = MultinomialNB()
modelo_nb_lexicon.fit(X_train_lexicon_normalizado, y_train)


predicciones_nb_lexicon = modelo_nb_lexicon.predict(X_test_lexicon_normalizado)


# Creamos y entrenamos el modelo de Regresión Logística con las características de lexicones
modelo_rl_lexicon = LogisticRegression(max_iter=1000)
modelo_rl_lexicon.fit(X_train_lexicon_normalizado, y_train)


predicciones_rl_lexicon = modelo_rl_lexicon.predict(X_test_lexicon_normalizado)



In [62]:
def evaluar_modelo(y_real, y_predicho, nombre_modelo, tipo_caracteristicas):

    #METRICAS    
    precision = precision_score(y_real, y_predicho, pos_label='positive')
    f1 = f1_score(y_real, y_predicho, pos_label='positive')
    recall = recall_score(y_real, y_predicho, pos_label='positive')
    exactitud = accuracy_score(y_real, y_predicho)
    
    print(f"{nombre_modelo} ({tipo_caracteristicas}):")
    
    print(f"  Precisión: {precision:.4f}")

    print(f"  Recall: {recall:.4f}")

    print(f"  F1 Score: {f1:.4f}")

    print(f"  Exactitud: {exactitud:.4f}")

    print()


evaluar_modelo(y_test,  predicciones_nb_lexicon, "NB", "Caracteristicas  de Lexicones")
evaluar_modelo(y_test, predicciones_rl_lexicon, "Regresión Logística",  "Características de Lexicones")



NB (Caracteristicas  de Lexicones):
  Precisión: 0.5364
  Recall: 0.9181
  F1 Score: 0.6772
  Exactitud: 0.5656

Regresión Logística (Características de Lexicones):
  Precisión: 0.7194
  Recall: 0.6877
  F1 Score: 0.7032
  Exactitud: 0.7119



In [63]:
nombres_features = tfidf_vectorizer.get_feature_names_out()
pesos_coef = lr_model_tfidf.coef_.flatten()

serie_pesos = pd.Series(data=pesos_coef, index=nombres_features)

#10 características con mayor peso positivo
top_positivas = serie_pesos.nlargest(10)
print("Top 10 palabras con mayor peso positivo:")
print(top_positivas)

# Seleccionamos las 10 características con mayor peso negativo
top_negativas = serie_pesos.nsmallest(10)
print("\nTop 10 palabras con mayor peso negativo:")
print(top_negativas)

# Obtenemos los nombres de las características y los coeficientes del modelo LR con lexicones
nombres_lexicon = lexicon_features_train.columns
pesos_lexicon = modelo_rl_lexicon.coef_.flatten()

# Creamos un DataFrame para visualizar los coeficientes
df_pesos_lexicon = pd.DataFrame({'Característica': nombres_lexicon,'Peso': pesos_lexicon})

print("\n Coeficientes del modelo de Regresión Logística con características de lexicones:")
print(df_pesos_lexicon)



Top 10 palabras con mayor peso positivo:
great        6.640917
excellent    4.949589
best         4.126957
easy         3.965901
love         3.591077
perfect      3.078033
price        2.770355
highly       2.649531
and          2.643829
well         2.572538
dtype: float64

Top 10 palabras con mayor peso negativo:
not              -5.905595
waste            -4.334232
worst            -3.813402
bad              -3.794728
poor             -3.471028
disappointed     -3.307788
boring           -3.049634
after            -2.963045
disappointing    -2.910968
disappointment   -2.908638
dtype: float64

 Coeficientes del modelo de Regresión Logística con características de lexicones:
       Característica       Peso
0      positive_count   2.484959
1      negative_count  -4.641186
2       neutral_count  -0.893440
3         afinn_score  11.699869
4  sentiwordnet_score   0.000000
